# Checking RT-Scheduled Trips Table 

In [1]:
import utils

from siuba import *
import pandas as pd

import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(130_000_000_000)

from calitp_data_analysis.sql import query_sql


/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:124: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
df = query_sql(f'''
SELECT * FROM `cal-itp-data-infra-staging.natalie_views.gtfs_rt_vs_schedule_trips_transitcenter_sample`
''')

In [3]:
df['service_date'] = pd.to_datetime(df['service_date'])
df['weekday'] = pd.Series(df.service_date).dt.day_name()    
df['month'] =  pd.Series(df.service_date).dt.month_name()

In [4]:
df.head()

,calitp_itp_id,agency_name,calitp_url_number,route_id,route_short_name,service_date,calitp_extracted_at,calitp_deleted_at,num_sched,num_vp,pct_w_vp,weekday,month
0,282,San Francisco Municipal Transportation Agency,1,M,M,2022-09-17,2022-02-16,2099-01-01,164,0,0.00,Saturday,September
1,282,San Francisco Municipal Transportation Agency,1,M,M,2022-09-18,2022-02-16,2099-01-01,164,0,0.00,Sunday,September
2,4,AC TRANSIT,0,U,U,2022-08-11,2022-03-02,2099-01-01,11,0,0.00,Thursday,August
3,4,AC TRANSIT,0,U,U,2022-09-06,2022-03-02,2099-01-01,11,0,0.00,Tuesday,September
4,4,AC TRANSIT,1,U,U,2022-08-15,2022-03-01,2099-01-01,11,0,0.00,Monday,August


## Check URLs

In [10]:
(utils.get_agg_pct(df,
                groupings= ['calitp_itp_id','agency_name', 'calitp_url_number'],
                sum_sched='num_sched',
                sum_vp='num_vp',
               ))>>arrange(_.agency_name, _.calitp_url_number)

,calitp_itp_id,agency_name,calitp_url_number,num_sched,num_vp,avg
1,4,AC TRANSIT,0,313342,256452,0.82
2,4,AC TRANSIT,1,313342,256436,0.82
3,278,MTS,0,226709,199510,0.88
4,278,San Diego International Airport,0,2040,0,0.00
5,282,San Francisco Municipal Transportation Agency,0,538884,0,0.00
0,282,San Francisco Municipal Transportation Agency,1,538884,429604,0.80


To drop:
* AC Transit url:1?
* San Diego International Airport url: 0
* SFMTA url: 0

In [11]:
table = (utils.get_correct_url(df))

In [12]:
(utils.get_agg_pct(table,
                groupings= ['calitp_itp_id','agency_name', 'calitp_url_number'],
                sum_sched='num_sched',
                sum_vp='num_vp',
               ))>>arrange(_.agency_name, _.calitp_url_number)

,calitp_itp_id,agency_name,calitp_url_number,num_sched,num_vp,avg
1,4,AC TRANSIT,1,313342,256436,0.82
2,278,MTS,0,226709,199510,0.88
0,282,San Francisco Municipal Transportation Agency,1,538884,429604,0.80


In [13]:
(utils.get_agg_pct(table,
                groupings= ['calitp_itp_id','agency_name', 'month'],
                sum_sched='num_sched',
                sum_vp='num_vp',
               ))>>arrange(_.agency_name, _.month)

,calitp_itp_id,agency_name,month,num_sched,num_vp,avg
1,4,AC TRANSIT,August,159194,129725,0.81
2,4,AC TRANSIT,September,154148,126711,0.82
5,278,MTS,August,206889,182020,0.88
4,278,MTS,September,19820,17490,0.88
3,282,San Francisco Municipal Transportation Agency,August,274758,217202,0.79
0,282,San Francisco Municipal Transportation Agency,September,264126,212402,0.80


In [16]:
## table.to_csv('gtfs_vehicle_positions_vs_scheduled_trips_sample_table.csv')

## Add Alerts

In [40]:
# def get_alert_data():
#     alerts = query_sql(
#         f"""
#         SELECT * FROM `cal-itp-data-infra.staging.stg_rt__service_alerts`
#         WHERE date BETWEEN '2022-09-01' AND '2022-09-30'
#         AND effect = 'NO_SERVICE'
#         AND (calitp_itp_id in (4))
#                 """)
#     return alerts
                

In [41]:
# alerts = get_alert_data()

In [42]:
## hit query limit
# alerts

## Add Trip Updates

In [3]:
analysis_date1 = '2022-09-01'
analysis_date2 = '2022-09-30'
# itp_ids = [4, 278, 282]


### ITP ID 282

In [ ]:
## SF Muni, itp id 282, shows trip updates as valid but does not return any results

In [17]:
## throwing an error
updates_282 = query_sql(f'''
SELECT *
FROM `cal-itp-data-infra.staging.stg_rt__trip_updates`
WHERE date BETWEEN '{analysis_date1}' AND '{analysis_date2}'
AND calitp_itp_id = 282 AND calitp_url_number = 1 AND
trip_schedule_relationship <> 'SCHEDULED'
LIMIT 2000
''')

DatabaseError: (google.cloud.bigquery.dbapi.exceptions.DatabaseError) 500 Query exceeded limit for bytes billed: 800000000000. 802703802368 or higher required.

Location: us-west2
Job ID: a872b316-23a4-4873-b340-a8725a93f670

[SQL: 
SELECT *
FROM `cal-itp-data-infra.staging.stg_rt__trip_updates`
WHERE date BETWEEN '2022-09-01' AND '2022-09-30'
AND calitp_itp_id = 282 AND calitp_url_number = 1 AND
trip_schedule_relationship <> 'SCHEDULED'
LIMIT 2000]
(Background on this error at: https://sqlalche.me/e/14/4xp6)

In [ ]:
# updates_282

### ITP ID 4

In [18]:
## AC transit, itp id 4, returns trip updates

In [12]:


updates_4 = query_sql(f'''
SELECT *
FROM `cal-itp-data-infra-staging.natalie_staging.stg_rt__trip_updates`
WHERE date BETWEEN '{analysis_date1}' AND '{analysis_date2}'
AND calitp_itp_id  = 4 AND calitp_url_number = 1 AND 
trip_schedule_relationship <> 'SCHEDULED'
LIMIT 5000
''')

In [13]:
len(updates_4)

5000

In [14]:
updates_4.sample(5)

,calitp_itp_id,calitp_url_number,original_file_path,date,id,timestamp,delay,vehicle_id,vehicle_label,vehicle_license_plate,trip_id,trip_route_id,trip_direction_id,trip_start_time,trip_start_date,trip_schedule_relationship,stop_time_updates,key
1725,4,1,gtfs-data/rt/2022-09-02T17:08:05/4/1/gtfs_rt_t...,2022-09-02,-835425274,1662138462,None,1601,,,-835425274,45,None,None,None,ADDED,"[{'stopSequence': 40, 'stopId': '57221', 'arri...",b3341899e122d202d79ccbb164b13631
4454,4,1,gtfs-data/rt/2022-09-03T16:49:37/4/1/gtfs_rt_t...,2022-09-03,-1861931827,1662223751,None,6138,,,-1861931827,800,None,None,None,ADDED,"[{'stopSequence': 9, 'stopId': '51969', 'arriv...",e3067be1936bf5460c2b51a4db0b0d2c
2561,4,1,gtfs-data/rt/2022-09-01T20:38:32/4/1/gtfs_rt_t...,2022-09-01,-1609608939,1662064688,None,2235,,,-1609608939,57,None,None,None,ADDED,"[{'stopSequence': 43, 'stopId': '56785', 'arri...",6a72036aa66b687d36d2a87c6f04d4c5
2427,4,1,gtfs-data/rt/2022-09-01T17:53:44/4/1/gtfs_rt_t...,2022-09-01,-1438823873,1662054799,None,2235,,,-1438823873,57,None,None,None,ADDED,"[{'stopSequence': 5, 'stopId': '51637', 'arriv...",d8345d83e81fc100c7642dec8c31bb8b
2882,4,1,gtfs-data/rt/2022-09-01T18:58:27/4/1/gtfs_rt_t...,2022-09-01,-1331188665,1662058684,None,5118,,,-1331188665,65,None,None,None,ADDED,"[{'stopSequence': 31, 'stopId': '55264', 'arri...",8edc394d1c565d83f04f7881585e4041


In [15]:
updates_4>>group_by(_.date, _.trip_route_id, _.trip_id) >>count(_.trip_schedule_relationship)>>arrange(_.date ,-_.trip_route_id)

,date,trip_route_id,trip_id,trip_schedule_relationship,n
25,2022-09-01,V,-689583214,ADDED,15
24,2022-09-01,FS,-1985747130,ADDED,23
22,2022-09-01,79,-1630764724,ADDED,98
23,2022-09-01,79,-526346110,ADDED,10
20,2022-09-01,7,-2054354814,ADDED,110
21,2022-09-01,7,-665387930,ADDED,107
18,2022-09-01,67,-1285126014,ADDED,46
19,2022-09-01,67,-1807440500,ADDED,65
14,2022-09-01,65,-1331188665,ADDED,92
15,2022-09-01,65,-1570165831,ADDED,9


In [16]:
(updates_4
 >>filter(_.trip_schedule_relationship != 'SCHEDULED')
 >>group_by(_.date, _.trip_route_id, _.trip_schedule_relationship)
 >>summarize(n = _.trip_id.nunique())
 >>arrange(-_.trip_route_id))


,date,trip_route_id,trip_schedule_relationship,n
11,2022-09-01,V,ADDED,1
20,2022-09-02,P,ADDED,1
29,2022-09-04,NL,ADDED,1
10,2022-09-01,FS,ADDED,1
19,2022-09-02,99,ADDED,2
28,2022-09-03,99,ADDED,1
27,2022-09-03,88,ADDED,1
26,2022-09-03,851,ADDED,1
30,2022-09-06,851,ADDED,1
18,2022-09-02,800,ADDED,2


* final table might look like the above cell

In [17]:
updates_4.trip_schedule_relationship.value_counts()

ADDED       4985
CANCELED      15
Name: trip_schedule_relationship, dtype: int64

In [18]:
updates_4>>filter(_.trip_schedule_relationship=='ADDED')>>count(_.trip_id)

,trip_id,n
0,-1105804378,81
1,-1156471184,31
2,-1180475466,28
3,-1189085104,176
4,-1211560501,149
5,-1285126014,46
6,-1331188665,92
7,-1402333184,238
8,-1438823873,34
9,-1469129356,159


### ITP ID 278

In [21]:
## MTS, itp id 278, has no valid trip updates 

In [19]:
## throwing an error
# updates_278 = query_sql(f'''
# SELECT *
# FROM `cal-itp-data-infra-staging.natalie_staging.stg_rt__trip_updates`
# WHERE date BETWEEN '{analysis_date1}' AND '{analysis_date2}'
# AND calitp_itp_id  =  278 AND trip_schedule_relationship <> 'SCHEDULED'
# LIMIT 50
# ''')

In [20]:
# updates_278

In [21]:
## SQL query to agg trip updates
# query_sql(f'''
# SELECT 
#   calitp_itp_id,
#   calitp_url_number,
#   date,
#   trip_route_id,
#   trip_schedule_relationship,
# COUNT(DISTINCT trip_id) AS Count,
# FROM `cal-itp-data-infra.staging.stg_rt__trip_updates` 
# WHERE calitp_itp_id = 4 AND calitp_url_number = 1 AND
#   date BETWEEN '2022-09-01' AND '2022-09-30' AND
#   trip_schedule_relationship <> 'SCHEDULED'
# GROUP BY calitp_itp_id, calitp_url_number, date, trip_route_id, trip_schedule_relationship
# ''')